In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [3]:
item_categories = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/item_categories.csv')
items = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/items.csv')
sales_train = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/sales_train.csv')
sample = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/sample_submission.csv')
shops = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/shops.csv')
test = pd.read_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/test.csv')

In [4]:
test.shape

(214200, 10)

# Memory

In [5]:
item_categories['item_category_id'] = item_categories['item_category_id'].astype('uint8')
items['item_id'] = items['item_id'].astype('uint16')
items['item_category_id'] = items['item_category_id'].astype('uint8')
sales_train['date_block_num'] = sales_train['date_block_num'].astype('uint8')
sales_train['shop_id'] = sales_train['shop_id'].astype('uint8')
sales_train['item_id'] = sales_train['item_id'].astype('uint16')
sales_train['item_cnt_day'] = sales_train['item_cnt_day'].astype('int32')
shops['shop_id'] = shops['shop_id'].astype('uint8')
test['shop_id'] = test['shop_id'].astype('uint8')
test['item_id'] = test['item_id'].astype('uint16')

In [6]:
sales_train.drop_duplicates(inplace=True)
sales_train['month'] = sales_train['date'].str.slice(start=3,stop=5).astype('uint8')
sales_train['year'] = sales_train['date'].str.slice(start=6).astype('uint16')
sales_train['day'] = sales_train['date'].str.slice(stop=2).astype('uint8')
sales_train.drop('date', axis=1, inplace=True)

In [7]:
shops.loc[shops['shop_id']==11, 'shop_id'] = 10
shops.loc[shops['shop_id']==0, 'shop_id'] = 57
shops.loc[shops['shop_id']==1, 'shop_id'] = 58
shops.loc[shops['shop_id']==23, 'shop_id'] = 24
shops.drop(index=[0, 1, 10, 23], inplace=True)
sales_train.loc[sales_train['shop_id']==11, 'shop_id'] = 10
sales_train.loc[sales_train['shop_id']==0, 'shop_id'] = 57
sales_train.loc[sales_train['shop_id']==1, 'shop_id'] = 58
sales_train.loc[sales_train['shop_id']==23, 'shop_id'] = 24
test.loc[test['shop_id']==11, 'shop_id'] = 10
test.loc[test['shop_id']==0, 'shop_id'] = 57
test.loc[test['shop_id']==1, 'shop_id'] = 58
test.loc[test['shop_id']==23, 'shop_id'] = 24

In [8]:
shops['city'] = ''
for i, el in enumerate(list(shops['shop_name'].str.split(' '))):
    shops['city'].iloc[i] = el[0]
city = dict(zip(list(shops['shop_id']), list(shops['city'])))
sales_train['city'] = sales_train['shop_id'].map(city)
test['city'] = test['shop_id'].map(city)


item_categories['split'] = item_categories['item_category_name'].str.split('-')
item_categories['type'] = item_categories['split'].map(lambda x: x[0].strip())
item_categories['sub_type'] = item_categories['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
item_categories.drop('split',axis=1,inplace=True)

d = dict(zip(list(item_categories['item_category_id']), list(item_categories['type'])))
x = dict(zip(list(item_categories['item_category_id']), list(item_categories['sub_type'])))
items['type'] = items['item_category_id'].map(d)
items['sub_type'] = items['item_category_id'].map(x)
cat = dict(zip(list(items['item_id']),list(items['item_category_id'])))
sales_train['item_category_id'] = sales_train['item_id'].map(cat)
sales_train['type'] = sales_train['item_category_id'].map(d)
sales_train['sub_type'] = sales_train['item_category_id'].map(x)
test['item_category_id'] = test['item_id'].map(cat)
test['type'] = test['item_category_id'].map(d)
test['sub_type'] = test['item_category_id'].map(x)
sales_train = sales_train[sales_train['item_price']<100000]
sales_train = sales_train[sales_train['item_cnt_day']<1001]

C:\Users\Dindar\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
item_categories.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_item_categories.csv', index=False)
items.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_items.csv', index=False)
sales_train.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_sales_train.csv', index=False)
sample.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_sample_submission.csv', index=False)
shops.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_shops.csv', index=False)
test.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_test.csv', index=False)

# filling nulls(item_cnt_day)

In [10]:
ex = sales_train[sales_train['item_cnt_day']<0]
ex1 = sales_train[sales_train['item_cnt_day']>0]

In [11]:
q = []
for i in (ex.index):
    if sales_train.iloc[i]['item_cnt_day'] < 0:
        q.append(i)

In [12]:
len(q)

258

In [13]:
for i in q:
    d = sales_train['date_block_num'].iloc[i]
    it = sales_train['item_id'].iloc[i]
    s = sales_train['shop_id'].iloc[i]
    mean = ex1[(ex1['date_block_num']==d)&(ex1['item_id']==it)&(ex1['shop_id']==s)]['item_cnt_day'].mean()
    sales_train['item_cnt_day'].iloc[i] = mean

In [14]:
w = sales_train[sales_train['item_cnt_day'].isnull()].index.tolist()

In [15]:
len(w)

51

In [16]:
for i in w:
    it = sales_train['item_id'].iloc[i]
    c = sales_train['city'].iloc[i]
    mean = ex1[(ex1['item_id']==it)&(ex1['city']==c)]['item_cnt_day'].mean()
    sales_train['item_cnt_day'].iloc[i] = mean

In [17]:
e = sales_train[sales_train['item_cnt_day'].isnull()].index.tolist()

In [18]:
len(e)

29

In [19]:
for i in e:
    it = sales_train['item_id'].iloc[i]
    mean = ex1[ex1['item_id']==it]['item_cnt_day'].mean()
    sales_train['item_cnt_day'].iloc[i] = mean

In [20]:
i = sales_train[sales_train['item_cnt_day'].isnull()].index.tolist()

In [21]:
sales_train.drop(index=i, inplace=True)

In [22]:
sales_train[sales_train['item_price']<0]

,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year,day,city,item_category_id,type,sub_type
484683,4,32,2973,-1.0,1.0,5,2013,15,Москва,19,Игры,PS3


In [23]:
ex = sales_train[sales_train['item_price']>0]

In [24]:
mean = ex[(ex['date_block_num']==4)&(ex['item_id']==2973)&(ex['shop_id']==32)]['item_price'].mean()

In [25]:
sales_train.loc[sales_train.item_price<0, 'item_price'] = mean

In [26]:
ex[(ex['date_block_num']==4)&(ex['item_id']==2973)&(ex['shop_id']==32)]

,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year,day,city,item_category_id,type,sub_type
484682,4,32,2973,1249.0,1.0,5,2013,23,Москва,19,Игры,PS3
484684,4,32,2973,2499.0,1.0,5,2013,6,Москва,19,Игры,PS3


In [27]:
sales_train[(sales_train['date_block_num']==4)&(sales_train['item_id']==2973)&(sales_train['shop_id']==32)]

,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year,day,city,item_category_id,type,sub_type
484682,4,32,2973,1249.0,1.0,5,2013,23,Москва,19,Игры,PS3
484683,4,32,2973,1874.0,1.0,5,2013,15,Москва,19,Игры,PS3
484684,4,32,2973,2499.0,1.0,5,2013,6,Москва,19,Игры,PS3


In [28]:
item_categories.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_item_categories.csv', index=False)
items.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_items.csv', index=False)
sales_train.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_sales_train.csv', index=False)
sample.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_sample_submission.csv', index=False)
shops.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_shops.csv', index=False)
test.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_test.csv', index=False)

# main grouped table

In [29]:
i = sales_train.groupby(['date_block_num','shop_id','item_id']).sum()['item_cnt_day'].index
df = i.to_frame(index = False)
df['item_cnt_month'] = list(sales_train.groupby(['date_block_num','shop_id','item_id']).sum()['item_cnt_day'])
df['item_cnt_month'] = df['item_cnt_month'].clip(0,20)
df['item_cnt_month'] = df['item_cnt_month'].fillna(0)
df = df[['date_block_num','shop_id','item_id','item_cnt_month']]
df.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [30]:
a =  [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
b = [1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
     1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
     1,  2,  3,  4,  5,  6,  7,  8,  9, 10]
y = dict(zip(a,b))
df['month'] = df['date_block_num'].map(y)

In [31]:
a =  [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
b = [2013,  2013,  2013,  2013,  2013,  2013,  2013,  2013,  2013, 2013, 2013, 2013,
     2014,  2014,  2014,  2014,  2014,  2014,  2014,  2014,  2014, 2014, 2014, 2014,
     2015,  2015,  2015,  2015,  2015,  2015,  2015,  2015,  2015, 2015]
y = dict(zip(a,b))
df['year'] = df['date_block_num'].map(y)

In [32]:
df.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_df.csv', index=False)

# test data

In [33]:
test['date_block_num'] = 34
test['year'] = 2015
test['month'] = 11

# adding city, cat....

In [34]:
df['city'] = df['shop_id'].map(city)
df['item_category_id'] = df['item_id'].map(cat)

In [35]:
d = dict(zip(list(item_categories['item_category_id']), list(item_categories['type'])))
s = dict(zip(list(item_categories['item_category_id']), list(item_categories['sub_type'])))

In [36]:
df['type'] = df['item_category_id'].map(d)
df['sub_type'] = df['item_category_id'].map(s)

In [37]:
df.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_df.csv', index=False)
test.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/test.csv', index=False)

In [38]:
df.shape, test.shape

((1607604, 10), (214200, 10))

# concat

In [39]:
data = pd.concat([df, test], ignore_index=True, sort=False)

In [40]:
data.fillna(0, inplace=True)

In [41]:
data.isnull().sum()

date_block_num      0
shop_id             0
item_id             0
item_cnt_month      0
month               0
year                0
city                0
item_category_id    0
type                0
sub_type            0
ID                  0
dtype: int64

In [42]:
data

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,ID
0,0.0,2,27,1.0,1,2013,Адыгея,19,Игры,PS3,0.0
1,0.0,2,33,1.0,1,2013,Адыгея,37,Кино,Blu,0.0
2,0.0,2,317,1.0,1,2013,Адыгея,45,Книги,Аудиокниги 1С,0.0
3,0.0,2,438,1.0,1,2013,Адыгея,45,Книги,Аудиокниги 1С,0.0
4,0.0,2,471,2.0,1,2013,Адыгея,49,Книги,Методические материалы 1С,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1821799,34.0,45,18454,0.0,11,2015,Самара,55,Музыка,CD локального производства,214195.0
1821800,34.0,45,16188,0.0,11,2015,Самара,64,Подарки,Настольные игры,214196.0
1821801,34.0,45,15757,0.0,11,2015,Самара,55,Музыка,CD локального производства,214197.0
1821802,34.0,45,19648,0.0,11,2015,Самара,40,Кино,DVD,214198.0


In [43]:
data.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data.csv', index=False)

In [44]:
data

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,ID
0,0.0,2,27,1.0,1,2013,Адыгея,19,Игры,PS3,0.0
1,0.0,2,33,1.0,1,2013,Адыгея,37,Кино,Blu,0.0
2,0.0,2,317,1.0,1,2013,Адыгея,45,Книги,Аудиокниги 1С,0.0
3,0.0,2,438,1.0,1,2013,Адыгея,45,Книги,Аудиокниги 1С,0.0
4,0.0,2,471,2.0,1,2013,Адыгея,49,Книги,Методические материалы 1С,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1821799,34.0,45,18454,0.0,11,2015,Самара,55,Музыка,CD локального производства,214195.0
1821800,34.0,45,16188,0.0,11,2015,Самара,64,Подарки,Настольные игры,214196.0
1821801,34.0,45,15757,0.0,11,2015,Самара,55,Музыка,CD локального производства,214197.0
1821802,34.0,45,19648,0.0,11,2015,Самара,40,Кино,DVD,214198.0


In [46]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [47]:
data = lag_feature(data, [1,2,3,6,12], 'item_cnt_month')

In [48]:
data = data.fillna(0)

In [49]:
data.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data.csv', index=False)

In [50]:
data_encoded = data.copy()

In [51]:
data

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,ID,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12
0,0.0,2,27,1.0,1,2013,Адыгея,19,Игры,PS3,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2,33,1.0,1,2013,Адыгея,37,Кино,Blu,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,2,317,1.0,1,2013,Адыгея,45,Книги,Аудиокниги 1С,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2,438,1.0,1,2013,Адыгея,45,Книги,Аудиокниги 1С,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2,471,2.0,1,2013,Адыгея,49,Книги,Методические материалы 1С,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821799,34.0,45,18454,0.0,11,2015,Самара,55,Музыка,CD локального производства,214195.0,1.0,0.0,0.0,0.0,0.0
1821800,34.0,45,16188,0.0,11,2015,Самара,64,Подарки,Настольные игры,214196.0,0.0,0.0,0.0,0.0,0.0
1821801,34.0,45,15757,0.0,11,2015,Самара,55,Музыка,CD локального производства,214197.0,0.0,0.0,0.0,0.0,0.0
1821802,34.0,45,19648,0.0,11,2015,Самара,40,Кино,DVD,214198.0,0.0,0.0,0.0,0.0,0.0


In [52]:
data_encoded['city'], _ = pd.factorize(data_encoded['city'])
data_encoded['type'], _ = pd.factorize(data_encoded['type'])

In [53]:
data_encoded.to_csv('C:/Users/Dindar/competitive-data-science-predict-future-sales(1)/tables/main_all_data_encoded.csv', index=False)

In [54]:
data_encoded.dtypes

date_block_num           float64
shop_id                   uint64
item_id                   uint64
item_cnt_month           float64
month                      int64
year                       int64
city                       int64
item_category_id           int64
type                       int64
sub_type                  object
ID                       float64
item_cnt_month_lag_1     float64
item_cnt_month_lag_2     float64
item_cnt_month_lag_3     float64
item_cnt_month_lag_6     float64
item_cnt_month_lag_12    float64
dtype: object

In [55]:
data_encoded

,date_block_num,shop_id,item_id,item_cnt_month,month,year,city,item_category_id,type,sub_type,ID,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12
0,0.0,2,27,1.0,1,2013,0,19,0,PS3,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2,33,1.0,1,2013,0,37,1,Blu,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,2,317,1.0,1,2013,0,45,2,Аудиокниги 1С,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2,438,1.0,1,2013,0,45,2,Аудиокниги 1С,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2,471,2.0,1,2013,0,49,2,Методические материалы 1С,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821799,34.0,45,18454,0.0,11,2015,17,55,6,CD локального производства,214195.0,1.0,0.0,0.0,0.0,0.0
1821800,34.0,45,16188,0.0,11,2015,17,64,5,Настольные игры,214196.0,0.0,0.0,0.0,0.0,0.0
1821801,34.0,45,15757,0.0,11,2015,17,55,6,CD локального производства,214197.0,0.0,0.0,0.0,0.0,0.0
1821802,34.0,45,19648,0.0,11,2015,17,40,1,DVD,214198.0,0.0,0.0,0.0,0.0,0.0
